In [ ]:
%%time
import pandas as pd
import fnmatch

p_ = list(pd.read_csv('../input/santa-2021/permutations.csv')['Permutation'].values)
p2_ = {p:0 for p in p_}
psa = {p:i for i,p in enumerate(p_) if p.startswith('🎅🤶')}
ps = {p:i for i,p in enumerate(p_) if not p.startswith('🎅🤶')}

f = open('submission.csv', 'w', encoding='utf-16')
f.write('schedule\n')

best_ = '🎅🤶🦌🧝🎄🎁🎀'
while len(p2_) > 0:
    for i in range(7,-1,-1):
        b = False
        suf = [p for p in p2_ if p.startswith(best_[-i:])]
        if len(suf)>0:
            best_ += suf[0][i:]
            p2_.pop(suf[0], None)
            b = True
        if b: break
        
psa = [p for p in p_ if p.startswith('🎅🤶')]

def get_chunk(best, start=0, stop=6000):
    target1 = 2549 
    target2 = 120
    besti = 0
    bestj = 0

    for i in range(start,len(best)-1000):
        for j in range(i+1000, len(best)):
            superp_ = best[i:j]
            target2_ = len([p for p in psa if p not in superp_])
            target1_ = len(superp_) + (target2_ * 7)
            if target1_ < target1 and target1_ > target1 - 14 and target2 > target2_:
                target1 = int(target1_)
                target2 = int(target2_)
                besti = int(i)
                bestj = int(j)
                print(i,j,target1, target2)
            if len(superp_)>(target1 + (target2 * 7)) :
                break
        if i % 100 == 0: print(i)
        if i  > stop: break
    return besti, bestj

i, j = get_chunk(best_, 0, 100)
part1 = best_[max(0,i-6):j+6]
new = best_[:i] + best_[j:]
i, j = get_chunk(new, 0, 100)
part2 = new[max(0,i-6):j+6]
part3 = new[:i] + new[j:]
print(len(part1), len(part2), len(part3))
parts = [part1, part2, part3]

for i in range(len(parts)):
    wilds = 2
    missing_required = [p for p in psa if p not in parts[i]]
    for j in range(7):
        matches = [p for p in missing_required if fnmatch.fnmatch(parts[i], '*' + p.replace(p[j],'?') + '*')]
        print(j, matches)
        for m in matches:
            if wilds > 0:
                wilds -= 1
                missing_required = [p for p in missing_required if p != m]
                for w in range(0,len(parts[i])-7):
                    if fnmatch.fnmatch(parts[i][w:w+7], m.replace(m[j],'?')):
                        print('---', parts[i][w:w+7])
                        parts[i] = parts[i][:w+j] + '🌟' + parts[i][w+j+1:]
                        print('***', parts[i][w:w+7])
    parts[i] += ''.join(missing_required)
    print(len(parts[i]))
    f.write(parts[i]+'\n')
f.close()

In [ ]:
from IPython.display import display, HTML
from itertools import permutations
import math

sequence = '🎅🤶🦌🧝🎄🎁🎀'
translator  = {sequence[i]: i+1 for i in range(len(sequence))}
best_ = ''.join([str(translator[best_[i]]) for i in range(len(best_))])
p_ = list(sorted([''.join(map(str, s)) for s in permutations([1, 2, 3, 4, 5, 6, 7])]))
psa = [p for p in p_ if p.startswith('12')]

order = {}
for i in range(0,len(best_)-7):
    p = best_[i:i+7]
    if p in p_:
        order[i] = [p,0,0]
        for j in range(len(p_)):
            if p_[j]==p:
                order[i][1] = j
        if p in psa:
            order[i][2] = 1
    else:
        order[i] = ['xxxxxxx','xxxx','x']

html = ['','','','','']
html[0] = '<table><tr><td><strong>Super Permutation</strong></td>'
html[1] = '<tr><td><strong>Permutations Sort Order Index</strong></td>'
html[2] = '<tr><td><strong>Kaggle Required</strong></td>'
html[3] = '<tr><td><strong>Permutation Start</strong></td>'
html[4] = '<tr><td><strong>Index</strong></td>'
for i in range(len(best_)-7):
    html[0] += '<td>' + best_[i] + '</td>'
    html[1] += '<td>' + str(order[i][1]) + '</td>'
    html[2] += '<td>' + str(order[i][2]) + '</td>'
    html[3] += '<td>' + order[i][0] + '</td>'
    html[4] += '<td>' + str(i) + '</td>'
for i in range(len(html)):
    html[i] += '</tr>'
html = '\n'.join(html) + '</table>'
html += '<br><strong>Min:</strong> ' + str(math.factorial(7) + 6) + ' < ' + str(len(best_)) + ' < ' + str(math.factorial(7) * 7) + ' <strong>:Max</strong>'
HTML(html)